In [9]:
import numpy as np
import pandas as pd
import yfinance as yf

from arch import arch_model

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error

import plotly.graph_objects as go
import plotly.express as px

In [10]:
ticker = "MRVL"
start_date = "2020-01-01"

data = yf.download(ticker, start=start_date, auto_adjust=True)
data.head(10)

[*********************100%***********************]  1 of 1 completed


Price,Close,High,Low,Open,Volume
Ticker,MRVL,MRVL,MRVL,MRVL,MRVL
Date,,,,,
2020-01-02,26.667011,26.676751,26.121594,26.296907,6503200
2020-01-03,25.975502,26.355346,25.780709,25.917065,9732300
2020-01-06,25.011284,25.615138,24.923627,25.605400,10432100
2020-01-07,25.566439,25.702792,25.118418,25.196335,7493800
2020-01-08,25.371645,25.741751,25.235291,25.556698,7860800
2020-01-09,25.420343,25.780707,25.196333,25.780707,8521300
2020-01-10,25.809929,25.907325,25.361908,25.517742,7200900
2020-01-13,26.803370,26.822849,25.956024,26.033941,11619100


In [11]:
prices = data["Close"]

returns = 100 * np.log(prices / prices.shift(1))
returns = returns.dropna()
returns = returns.squeeze()

returns.name = "r"
returns.head()

Date
2020-01-03   -2.627340
2020-01-06   -3.782679
2020-01-07    2.195342
2020-01-08   -0.764829
2020-01-09    0.191756
Name: r, dtype: float64

In [12]:
window = 10

realized_vol = returns.rolling(window).std() * np.sqrt(252)
realized_vol = realized_vol.shift(-1)
realized_vol.name = "rv_future"

dataset = pd.concat([returns, realized_vol], axis=1).dropna()
dataset.head()

,r,rv_future
Date,,
2020-01-15,-2.913738,47.842734
2020-01-16,5.568132,44.956227
2020-01-17,0.845966,44.190844
2020-01-21,-3.500360,43.444365
2020-01-22,0.976687,42.811277


In [13]:
dataset = pd.concat([returns, realized_vol], axis=1)
print(dataset.shape)
dataset.head()


(1484, 2)


,r,rv_future
Date,,
2020-01-03,-2.627340,NaN
2020-01-06,-3.782679,NaN
2020-01-07,2.195342,NaN
2020-01-08,-0.764829,NaN
2020-01-09,0.191756,NaN


GARCH: σ²ₜ = ω + α·ε²ₜ₋₁ + β·σ²ₜ₋₁

In [14]:
am = arch_model(returns, vol="GARCH", p=1, q=1, dist="normal")
garch_res = am.fit(disp="off")
print(garch_res.summary())

                     Constant Mean - GARCH Model Results                      
Dep. Variable:                      r   R-squared:                       0.000
Mean Model:             Constant Mean   Adj. R-squared:                  0.000
Vol Model:                      GARCH   Log-Likelihood:               -3955.23
Distribution:                  Normal   AIC:                           7918.46
Method:            Maximum Likelihood   BIC:                           7939.67
                                        No. Observations:                 1484
Date:                Thu, Nov 27 2025   Df Residuals:                     1483
Time:                        15:20:11   Df Model:                            1
                               Mean Model                               
                 coef    std err          t      P>|t|  95.0% Conf. Int.
------------------------------------------------------------------------
mu             0.0651  9.086e-02      0.717      0.473 [ -0.113,  0.24

In [15]:
garch_forecasts = garch_res.forecast(horizon=1, reindex=True)
garch_var = garch_forecasts.variance.iloc[:, 0]
garch_vol = garch_res.conditional_volatility * np.sqrt(252) # annualized
garch_vol.name = "garch_vol"

aligned = pd.concat([returns, realized_vol, garch_vol], axis=1).dropna()
aligned.head()

,r,rv_future,garch_vol
Date,,,
2020-01-15,-2.913738,47.842734,47.979781
2020-01-16,5.568132,44.956227,49.811521
2020-01-17,0.845966,44.190844,59.412620
2020-01-21,-3.500360,43.444365,54.529602
2020-01-22,0.976687,42.811277,55.780156


In [16]:
fig = px.line(aligned[["rv_future", "garch_vol"]], title=f"{ticker} – Realized vs GARCH Volatility"
)
fig.show()


GARCH reacts gradually to new market information due to the nature of the model. When realized volatility spikes unexpectedly, GARCH initially underreacts but then adjusts upward over the following days; when volatility fades, it tends to overreact and then gradually mean-revert. E.g. On May 8th, 2022 (SPY), we see a jump in realized volatility. GARCH naturally underreacts to this price action, overreacts on May 19th, and smooths itself out over the following period.

I define a volatility shock as any day where realized volatility exceeds the 95th percentile of its historical distribution. This avoids arbitrary thresholds and provides a clean way to identify unusually volatile periods.

In [17]:
def make_lagged(df, col, lags):
    df = df.copy()
    for L in range(1, lags + 1):
        df[f"{col}_lag{L}"] = df[col].shift(L)
    return df

df = aligned.dropna(subset=["r", "rv_future", "garch_vol"]).copy()

df = make_lagged(df, "r", lags=5)
df = make_lagged(df, "garch_vol", lags=5)

print("Before dropna:", df.shape)
print(df.isna().sum())   # see where NaNs remain

df = df.dropna()

print("After dropna:", df.shape)
df.head()


Before dropna: (1475, 13)
r                 0
rv_future         0
garch_vol         0
r_lag1            1
r_lag2            2
r_lag3            3
r_lag4            4
r_lag5            5
garch_vol_lag1    1
garch_vol_lag2    2
garch_vol_lag3    3
garch_vol_lag4    4
garch_vol_lag5    5
dtype: int64
After dropna: (1470, 13)


,r,rv_future,garch_vol,r_lag1,r_lag2,r_lag3,r_lag4,r_lag5,garch_vol_lag1,garch_vol_lag2,garch_vol_lag3,garch_vol_lag4,garch_vol_lag5
Date,,,,,,,,,,,,,
2020-01-23,0.860203,45.907714,51.899991,0.976687,-3.500360,0.845966,5.568132,-2.913738,55.780156,54.529602,59.412620,49.811521,47.979781
2020-01-24,-2.566539,53.903984,48.949203,0.860203,0.976687,-3.500360,0.845966,5.568132,51.899991,55.780156,54.529602,59.412620,49.811521
2020-01-27,-5.416902,50.544146,49.618941,-2.566539,0.860203,0.976687,-3.500360,0.845966,48.949203,51.899991,55.780156,54.529602,59.412620
2020-01-28,1.762496,54.986916,59.217133,-5.416902,-2.566539,0.860203,0.976687,-3.500360,49.618941,48.949203,51.899991,55.780156,54.529602
2020-01-29,-4.823978,53.901772,55.277002,1.762496,-5.416902,-2.566539,0.860203,0.976687,59.217133,49.618941,48.949203,51.899991,55.780156


In [18]:
feature_cols = [c for c in df.columns if c.startswith("r_") or c.startswith("garch_vol_")]
X = df[feature_cols].values
y = df["rv_future"].values

len(df), X.shape, y.shape

(1470, (1470, 10), (1470,))

In [19]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, shuffle = False
)

print("Training samples:", len(X_train), "Testing samples:", len(X_test))

Training samples: 1176 Testing samples: 294


In [20]:
linreg = LinearRegression()
linreg.fit(X_train, y_train)

y_pred_lr = linreg.predict(X_test)

mae_lr = mean_absolute_error(y_test, y_pred_lr)
rmse_lr = mean_squared_error(y_test, y_pred_lr)
mae_lr, rmse_lr

(10.715711017896128, 329.4497236745726)

In [21]:
coef_table = pd.DataFrame({
    "feature": feature_cols,
    "coef": linreg.coef_
}).sort_values(by="coef", key = abs, ascending = False)

coef_table.head(10)

,feature,coef
5,garch_vol_lag1,0.845013
9,garch_vol_lag5,0.216846
8,garch_vol_lag4,0.216843
7,garch_vol_lag3,0.204732
6,garch_vol_lag2,0.204022
0,r_lag1,0.189302
2,r_lag3,-0.109896
3,r_lag4,-0.054712
1,r_lag2,0.026746
4,r_lag5,-0.018069


In [22]:
rf = RandomForestRegressor(n_estimators=100, max_depth = 5, random_state=42)

rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)

mae_rf = mean_absolute_error(y_test, y_pred_rf)
rmse_rf = mean_squared_error(y_test, y_pred_rf)
mae_rf, rmse_rf

(10.813690383873453, 324.6120816780693)

In [23]:
importances = rf.feature_importances_
fi_table = pd.DataFrame({
    "feature": feature_cols,
    "importance": importances
}).sort_values(by="importance", ascending=False)

fi_table.head(10)

,feature,importance
5,garch_vol_lag1,0.711360
6,garch_vol_lag2,0.108600
7,garch_vol_lag3,0.077885
0,r_lag1,0.046890
8,garch_vol_lag4,0.029848
9,garch_vol_lag5,0.015126
1,r_lag2,0.003015
2,r_lag3,0.002997
4,r_lag5,0.002321
3,r_lag4,0.001960


In [29]:
test_index = df.index[-len(y_test):]

comparison = pd.DataFrame({
    "rv_future": df.loc[test_index, "rv_future"],
    "garch_vol": df.loc[test_index, "garch_vol"],
    "lr_pred": y_pred_lr,
    "rf_pred": y_pred_rf
})

comparison.tail()

,rv_future,garch_vol,lr_pred,rf_pred
Date,,,,
2025-11-19,50.482358,62.714079,44.128171,46.927431
2025-11-20,52.402431,60.647476,53.488442,59.453031
2025-11-21,68.585465,67.630483,52.460253,55.840758
2025-11-24,66.228749,60.884813,60.416757,63.789233
2025-11-25,72.192076,75.495848,60.665519,64.162163


In [25]:
def metrics(true, pred):
    return {
        "MAE": mean_absolute_error(true, pred),
        "RMSE": np.sqrt(mean_squared_error(true, pred))
    } 

results = pd.DataFrame({
    "GARCH": metrics(comparison["rv_future"], comparison["garch_vol"]),
    "LinReg": metrics(comparison["rv_future"], comparison["lr_pred"]),
    "RandomForest": metrics(comparison["rv_future"], comparison["rf_pred"]) 
}).T

results

,MAE,RMSE
GARCH,14.323942,20.613311
LinReg,10.715711,18.150750
RandomForest,10.813690,18.016994


In [26]:
fig = px.line(
    comparison[["rv_future", "garch_vol", "lr_pred", "rf_pred"]],
    title=f"{ticker} – Volatility Forecasts (Test Period)"
)
fig.update_yaxes(title_text="Volatility (Annualized %)")
fig.show()

In [27]:
horizon = 30  # days ahead

# In-sample + out-of-sample forecasts from the beginning
multi_fc = garch_res.forecast(horizon=horizon, start=0)  # <-- key change

var_surface = multi_fc.variance
vol_surface = np.sqrt(var_surface)   # daily σ_t(h)

# Drop initial presample rows that are NaN in *any* horizon
vol_surface = vol_surface.dropna(how="any")

# Optional: keep only last 180 origin dates so the plot isn't huge
vol_surface = vol_surface.iloc[-180:]
vol_surface.columns = np.arange(1, horizon + 1)

print(vol_surface.shape)
vol_surface.head()


(180, 30)


,1,2,3,4,5,6,7,8,9,10,...,21,22,23,24,25,26,27,28,29,30
Date,,,,,,,,,,,,,,,,,,,,,
2025-03-13,5.203020,5.037010,4.889198,4.757932,4.641654,4.538900,4.448306,4.368605,4.298627,4.237301,...,3.912540,3.901092,3.891184,3.882612,3.875198,3.868788,3.863247,3.858458,3.854320,3.850745
2025-03-14,4.553321,4.461008,4.379770,4.308422,4.245878,4.191146,4.143327,4.101606,4.065252,4.033612,...,3.869680,3.864018,3.859124,3.854896,3.851242,3.848087,3.845361,3.843006,3.840973,3.839217
2025-03-17,4.155751,4.112440,4.074689,4.041822,4.013239,3.988404,3.966844,3.948141,3.931927,3.917879,...,3.846066,3.843615,3.841499,3.839672,3.838094,3.836731,3.835555,3.834539,3.833663,3.832906
2025-03-18,3.952648,3.935833,3.921263,3.908644,3.897719,3.888266,3.880088,3.873016,3.866901,3.861616,...,3.834784,3.833873,3.833088,3.832409,3.831824,3.831319,3.830882,3.830506,3.830181,3.829900
2025-03-19,3.688605,3.708017,3.724690,3.739021,3.751346,3.761951,3.771080,3.778941,3.785713,3.791549,...,3.820916,3.821904,3.822757,3.823493,3.824129,3.824677,3.825150,3.825558,3.825910,3.826214


In [28]:
X_dates = vol_surface.index
Y_horizons = vol_surface.columns
Z = vol_surface.values * np.sqrt(252)  # annualize


X_grid = np.tile(X_dates.values, (len(Y_horizons), 1)).T
Y_grid = np.tile(Y_horizons.values, (len(X_dates), 1))

fig = go.Figure(
    data=[go.Surface(
        x=X_dates, y=Y_horizons, z=Z,
        colorscale="Turbo",
        opacity=0.95,
        colorbar=dict(
            tickfont=dict(color="white"),
            title=dict(text="Vol", font=dict(color="white"))
        )
    )]
)


fig.update_layout(
    # --- overall figure size & margins ---
    width=900,
    height=700,
    margin=dict(l=0, r=0, t=60, b=0),

    # --- background + global font ---
    paper_bgcolor="#000000",   # whole notebook cell background
    plot_bgcolor="#000000",    # background behind 3D scene
    font=dict(color="white"),

    # --- title ---
    title=dict(
        text=f"{ticker} – GARCH Volatility Forecast Surface",
        x=0.03,                 # left-ish
        xanchor="left",
        font=dict(size=24, color="white")
    ),

    # --- 3D scene styling ---
    scene=dict(
        bgcolor="rgba(0,0,0,0)",    # transparent cube background

        xaxis=dict(
            title=dict(
                text="Horizon (Days)",
                font=dict(size=14, color="white")
            ),
            tickfont=dict(color="white"),
            showgrid=False,
            showbackground=False,
            zeroline=False
        ),

        yaxis=dict(
            title=dict(
                text="Date",
                font=dict(size=14, color="white")
            ),
            tickfont=dict(color="white"),
            showgrid=False,
            showbackground=False,
            zeroline=False
        ),

        zaxis=dict(
            title=dict(
                text="Volatility (Annualized %)",
                font=dict(size=14, color="white")
            ),
            tickfont=dict(color="white"),
            showgrid=False,
            showbackground=False,
            zeroline=False
        ),
    ),

    # --- camera / perspective ---
    scene_camera=dict(
        eye=dict(x=1.6, y=1.6, z=0.9)   # tweak to taste
    )
)

fig.show()